In [3]:
#activo el entorno
! .NBA_venv\Scripts\activate 
#compuebo si estoy trabajando en el entorno
! where python

c:\Users\ERBC\Desktop\curso henry\Analisis_evolutivo_NBA\Analisis_evolutivo_NBA\.NBA_venv\Scripts\python.exe
C:\Users\ERBC\AppData\Local\Programs\Python\Python313\python.exe
C:\Users\ERBC\AppData\Local\Microsoft\WindowsApps\python.exe


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

ruta_guardar = r'C:\Users\ERBC\Desktop\curso henry\Analisis_evolutivo_NBA\Analisis_evolutivo_NBA\datasets_limpios\Estadisticas_jugadores.csv'

# Configurar Selenium
driver = webdriver.Chrome()
url = "https://www.nba.com/stats/alltime-leaders?SeasonType=Regular%20Season&PerMode=PerGame&StatCategory=PTS"
driver.get(url)

# Esperar a que la tabla cargue
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_table__p1iZz")))

# Extraer nombres de columnas automáticamente
header_elements = driver.find_elements(By.CSS_SELECTOR, "table thead tr th")
columns = [col.text.strip() for col in header_elements]

# Lista para almacenar los datos
all_stats = []

# Número de páginas a recorrer
MAX_PAGES = 28  # Hay 28 páginas en total
clicks_realizados = 0

while clicks_realizados < MAX_PAGES:
    # Extraer las filas de la tabla
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    
    for row in rows:
        columns_data = row.find_elements(By.TAG_NAME, "td")
        stats = [col.text for col in columns_data]  # Extraer el texto de cada columna
        if stats:
            all_stats.append(stats)

    # Intentar hacer clic en "Next Page" (hasta la 27 página)
    if clicks_realizados < MAX_PAGES - 1:
        try:
            next_button = driver.find_element(By.XPATH, "//button[@title='Next Page Button']")
            
            if "disabled" in next_button.get_attribute("class"):
                print("Botón deshabilitado. No hay más páginas.")
                break
            
            driver.execute_script("arguments[0].scrollIntoView();", next_button)  # Hacer scroll al botón
            driver.execute_script("arguments[0].click();", next_button)  # Forzar clic con JS
            time.sleep(3)  # Esperar que cargue la nueva página
            clicks_realizados += 1  # Contar el clic
        except Exception as e:
            print("No se encontró el botón o no se pudo hacer clic:", e)
            break
    else:
        break  # Salir después de la última página

# Cerrar navegador
driver.quit()

# Crear DataFrame con columnas dinámicas
df = pd.DataFrame(all_stats, columns=columns)


# Renombrar las columnas del DataFrame para que coincidan con la base de datos
df = df.rename(columns={
    '#': 'position',  
    'FG%': 'FG_PCT',
    '3PM': 'TPM',
    '3PA': 'TPA',
    '3P%': 'TP_PCT',
    'FT%': 'FT_PCT',
    'EFG%': 'EFG_PCT',
    'TS%': 'TS_PCT'

})


print(f"Scraping completado ({clicks_realizados + 1} páginas)")


Scraping completado (28 páginas) y guardado en nba_leaders.csv


In [9]:
# Convertir columnas numéricas que están como 'object' a tipos correctos
numeric_columns = ["position", "GP", "MIN", "PTS", "FGM", "FGA", "FG_PCT", 
                   "TPM", "TPA", "TP_PCT", "FTM", "FTA", "FT_PCT", 
                   "OREB", "DREB", "REB", "AST", "STL", "BLK", "TOV", 
                   "EFG_PCT", "TS_PCT"]

for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Convierte y reemplaza errores con NaN

# Reemplazar NaN con un valor predeterminado (por ejemplo, 0 si tiene sentido en tu contexto)
df.fillna(0, inplace=True)

# Guardar en CSV
df.to_csv(ruta_guardar, index=False)


In [10]:
# 1. Leer el dataset desde el archivo CSV
ruta_csv = r"C:\Users\ERBC\Desktop\curso henry\Analisis_evolutivo_NBA\Analisis_evolutivo_NBA\datasets_limpios\Estadisticas_jugadores.csv"
df = pd.read_csv(ruta_csv)
# Mostrar las primeras filas del dataset
print("Dataset cargado:")
df.info()

Dataset cargado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1381 entries, 0 to 1380
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   position  1381 non-null   int64  
 1   PLAYER    1381 non-null   object 
 2   GP        1381 non-null   int64  
 3   MIN       1381 non-null   float64
 4   PTS       1381 non-null   float64
 5   FGM       1381 non-null   float64
 6   FGA       1381 non-null   float64
 7   FG_PCT    1381 non-null   float64
 8   TPM       1381 non-null   float64
 9   TPA       1381 non-null   float64
 10  TP_PCT    1381 non-null   float64
 11  FTM       1381 non-null   float64
 12  FTA       1381 non-null   float64
 13  FT_PCT    1381 non-null   float64
 14  OREB      1381 non-null   float64
 15  DREB      1381 non-null   float64
 16  REB       1381 non-null   float64
 17  AST       1381 non-null   float64
 18  STL       1381 non-null   float64
 19  BLK       1381 non-null   float64
 20  TOV       138

In [11]:
import pyodbc
# 2. Conectar a la base de datos SQL Server
server = 'ERBC-GAMER\SQLEXPRESS'  # Nombre del servidor
database = 'NBA_database'  # Nombre de la base de datos
username = 'Sa'  # Usuario de SQL Server
password = '1234'  # Contraseña de SQL Server

# Cadena de conexión con usuario de sql
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

#conexion con usuario de windows
#conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Establecer la conexión
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
print("Conectado al server")

Conectado al server


<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
C:\Users\ERBC\AppData\Local\Temp\ipykernel_15420\3815601064.py:3: SyntaxWarning: invalid escape sequence '\S'
  server = 'ERBC-GAMER\SQLEXPRESS'  # Nombre del servidor


In [12]:
# Definir el nombre de la tabla
table_name = 'Estadisticas_jugadores'

# Consulta para verificar si la tabla ya existe
check_table_query = f'''
SELECT COUNT(*) 
FROM sys.tables 
WHERE name = '{table_name}';
'''

cursor.execute(check_table_query)
table_exists = cursor.fetchone()[0]  # Devuelve 1 si la tabla existe, 0 si no.

if table_exists == 0:
    # Crear la tabla si no existe
    create_table_query = f'''
    CREATE TABLE {table_name} (
        position INT NOT NULL, -- Posición del jugador en el ranking
        PLAYER NVARCHAR(100) NOT NULL PRIMARY KEY, -- Nombre del jugador (clave primaria)
        GP INT NOT NULL, -- Juegos jugados
        MIN FLOAT NOT NULL, -- Minutos por partido
        PTS FLOAT NOT NULL, -- Puntos por partido
        FGM FLOAT NOT NULL, -- Tiros de campo anotados
        FGA FLOAT NOT NULL, -- Tiros de campo intentados
        FG_PCT FLOAT NOT NULL, -- Porcentaje de tiros de campo
        TPM FLOAT NULL, -- Triples anotados
        TPA FLOAT NULL, -- Triples intentados
        TP_PCT FLOAT NULL, -- Porcentaje de triples
        FTM FLOAT NOT NULL, -- Tiros libres anotados
        FTA FLOAT NOT NULL, -- Tiros libres intentados
        FT_PCT FLOAT NOT NULL, -- Porcentaje de tiros libres
        OREB FLOAT NULL, -- Rebotes ofensivos
        DREB FLOAT NULL, -- Rebotes defensivos
        REB FLOAT NOT NULL, -- Rebotes totales
        AST FLOAT NOT NULL, -- Asistencias por partido
        STL FLOAT NULL, -- Robos por partido
        BLK FLOAT NULL, -- Bloqueos por partido
        TOV FLOAT NULL, -- Pérdidas de balón por partido
        EFG_PCT FLOAT NOT NULL, -- Porcentaje de tiros de campo efectivo
        TS_PCT FLOAT NOT NULL  -- Porcentaje de tiros verdaderos
    );
    '''
    cursor.execute(create_table_query)
    conn.commit()
    print(f"Tabla '{table_name}' creada exitosamente.")
else:
    print(f"La tabla '{table_name}' ya existe.")


La tabla 'Estadisticas_jugadores' ya existe.


In [13]:
# Borrar los datos antes de insertar los nuevos (mantiene la tabla vacía y limpia)
cursor.execute(f"DELETE FROM {table_name};")
conn.commit()
print(f"Datos anteriores en '{table_name}' eliminados.")

# Convertir el DataFrame a una lista de tuplas
data_tuples = [tuple(row) for row in df.to_numpy()]

# Definir la consulta MERGE para evitar duplicados
insert_query = f'''
MERGE INTO {table_name} AS target
USING (VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)) AS source (
    position, PLAYER, GP, MIN, PTS, FGM, FGA, FG_PCT, TPM, TPA, TP_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, STL, BLK, TOV, EFG_PCT, TS_PCT
)
ON target.PLAYER = source.PLAYER
WHEN MATCHED THEN 
    UPDATE SET 
        position = source.position,
        GP = source.GP,
        MIN = source.MIN,
        PTS = source.PTS,
        FGM = source.FGM,
        FGA = source.FGA,
        FG_PCT = source.FG_PCT,
        TPM = source.TPM,
        TPA = source.TPA,
        TP_PCT = source.TP_PCT,
        FTM = source.FTM,
        FTA = source.FTA,
        FT_PCT = source.FT_PCT,
        OREB = source.OREB,
        DREB = source.DREB,
        REB = source.REB,
        AST = source.AST,
        STL = source.STL,
        BLK = source.BLK,
        TOV = source.TOV,
        EFG_PCT = source.EFG_PCT,
        TS_PCT = source.TS_PCT
WHEN NOT MATCHED BY TARGET THEN
    INSERT (position, PLAYER, GP, MIN, PTS, FGM, FGA, FG_PCT, TPM, TPA, TP_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, STL, BLK, TOV, EFG_PCT, TS_PCT)
    VALUES (source.position, source.PLAYER, source.GP, source.MIN, source.PTS, source.FGM, source.FGA, source.FG_PCT, source.TPM, source.TPA, source.TP_PCT, source.FTM, source.FTA, source.FT_PCT, source.OREB, source.DREB, source.REB, source.AST, source.STL, source.BLK, source.TOV, source.EFG_PCT, source.TS_PCT);
'''

# Ejecutar la inserción de datos
cursor.executemany(insert_query, data_tuples)
conn.commit()
print(f"Datos de '{table_name}' insertados/actualizados correctamente.")


Datos anteriores en 'Estadisticas_jugadores' eliminados.
Datos de 'Estadisticas_jugadores' insertados/actualizados correctamente.
